In [14]:
import os
import re
import tqdm
import string
import pandas as pd
import numpy as np
import util
from sklearn.decomposition import TruncatedSVD
from data_preprocessing import *
import word_embedding_load as wel
from classification import *
from xgboost_classifier import *
from testaccuracy import *

In [15]:
[all_data, train_size, test_size, train_x, train_y, test_x] = util.loadData()
sentences = data_preprocess(all_data)

In [16]:
Text_INPUT_DIM=200
GENE_INPUT_DIM=100
TEXT_INPUT_DIM=200

In [17]:
svd = TruncatedSVD(n_components=GENE_INPUT_DIM, n_iter=25, random_state=12)

In [18]:
truncated_one_hot_gene = wel.getGeneVec(all_data, svd)
truncated_one_hot_variation = wel.getVariationVec(all_data, svd)

In [19]:
train_set = np.hstack((truncated_one_hot_gene[:train_size], truncated_one_hot_variation[:train_size]))
test_set = np.hstack((truncated_one_hot_gene[train_size:], truncated_one_hot_variation[train_size:]))
encoded_y = pd.get_dummies(train_y)
encoded_y = np.array(encoded_y)
print(encoded_y.shape)

(3689, 9)


In [20]:
print(train_set.shape)
print(test_set.shape)
train_set[0, 25:50]

(3689, 200)
(986, 200)


array([  7.76916933e-24,  -3.53450521e-23,   5.31491802e-23,
         2.51339257e-21,  -3.63598663e-21,   2.30917952e-20,
        -7.39341277e-21,  -2.21096698e-20,   2.14299368e-20,
        -4.86843729e-21,  -5.92934867e-20,  -2.03106444e-22,
         1.54878336e-20,  -8.47291848e-21,   2.88034068e-22,
        -9.48737097e-23,  -2.63938359e-20,   6.08685031e-21,
         1.13193970e-21,  -1.54651858e-21,   1.03682071e-22,
        -5.84588211e-22,   9.40684492e-22,   1.07367049e-23,
         1.19962006e-23])

In [21]:
#the xgboost classfication model
#first deal with the input label, transfrom it from 1-9 to 0-8(required by the xgboost)
for i in range(len(train_y)):
    train_y[i] -=1  

In [22]:
y_predict = xgbclassifier(train_set, train_y, test_set, 1, 8, 1000)

[0]	train-mlogloss:2.12624	valid-mlogloss:2.14912
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
[25]	train-mlogloss:1.18483	valid-mlogloss:1.54605
[50]	train-mlogloss:0.774752	valid-mlogloss:1.33676
[75]	train-mlogloss:0.549671	valid-mlogloss:1.24813
[100]	train-mlogloss:0.400179	valid-mlogloss:1.21325
[125]	train-mlogloss:0.293736	valid-mlogloss:1.20167
[150]	train-mlogloss:0.217797	valid-mlogloss:1.19216
[175]	train-mlogloss:0.164637	valid-mlogloss:1.18945
[200]	train-mlogloss:0.126945	valid-mlogloss:1.19636
[225]	train-mlogloss:0.098692	valid-mlogloss:1.20344
[250]	train-mlogloss:0.078175	valid-mlogloss:1.21442
Stopping. Best iteration:
[174]	train-mlogloss:0.166182	valid-mlogloss:1.18937

final validation loss:
1.18936974097
final validation accuracy:
0.571428571429


In [23]:
savesubmisstion(y_predict, test_x, filename = "submission_allwin2loadgeneonly.csv")